In [68]:
import pandas as pd
from gurobipy import *
from datetime import timedelta



This function reads in the historical arrival data from "Data/Historical_Data.csv", and returns a dataframe in which each cell as been replaced by the number of minutes past 8 am.

In [69]:
def Read_Data():
    
    df = pd.read_csv("data/Historical_Data.csv")
    
    for col in df.columns:
        df[col] = pd.to_datetime(df[col]).apply(lambda x:\
                                    (x - (pd.Timestamp(x.date())\
                                + timedelta(hours=8))).seconds/60 )
        
   
    return df
    

In [70]:
df = Read_Data()
df.head()

,start,stop_1,stop_2,stop_3,stop_4,stop_5
0,0.0,4.0,14.0,33.0,39.0,52.0
1,0.0,5.0,14.0,35.0,41.0,52.0
2,0.0,5.0,16.0,34.0,42.0,54.0
3,0.0,3.0,13.0,32.0,40.0,53.0
4,0.0,6.0,16.0,36.0,43.0,54.0


# Swiftly Scheduling

Solve the IP to optimize the schedule, i.e. maximize the number of historical arrivals that would be considered on time (1 min early or 4 min late). Return the objective value and the first start time designated as afternoon (as the number of minutes past 8 am).

In [182]:

def Optimize_Schedule():
    df = Read_Data()
    
    M = 10000
    stops = [1,2,3,4,5]
    times = {5,8}
    cutoffs = {}
    cutoffs = df["start"].unique().tolist()
    vals = {}
    
    for c in cutoffs:
        m = Model()
        x = {}
        y = {}
        z = {}
        for index in df.index:
            for s in stops:
                x[s] = m.addVar(0,GRB.INFINITY,0,GRB.CONTINUOUS)
                y[s] = m.addVar(0,GRB.INFINITY,0,GRB.CONTINUOUS)
                z[(s,index)] = m.addVar(0,1,1,GRB.BINARY)
        for index in df.index:
            for s in stops:
                if df.loc[index,'start'] < c:
                    m.addConstr(df.loc[index,f'stop_{s}'] - df.loc[index,'start'] - x[s] - 4 <= 4 * (1 - z[(s,index)]) * M)
                    m.addConstr(df.loc[index,f'stop_{s}'] - df.loc[index,'start'] - x[s] + 1 >= -1 * (1 - z[(s,index)]) * M)
                else:
                    m.addConstr(df.loc[index,f'stop_{s}'] - df.loc[index,'start'] - y[s] - 4 <= 4 * (1 - z[(s,index)]) * M)
                    m.addConstr(df.loc[index,f'stop_{s}'] - df.loc[index,'start'] - y[s] + 1 >= -1 * (1 - z[(s,index)]) * M)
        m.setObjective(quicksum(z[(s,index)] for s in stops for index in df.index),GRB.MAXIMIZE)
        m.setParam("OutputFlag",False)
        m.optimize()
        vals[c] = m.objVal
    
    max_key = max(vals,key=vals.get)
    max_value = vals[max_key]
    print(max_value)
    print(max_key)
    
    return max_value,max_key
    
    
    

In [183]:
#This might take a minute or two to run

num_ot, last_morning = Optimize_Schedule()
assert num_ot==1421
assert last_morning == 255

1421.0
255.0
